In [16]:
! ls /gpfs/gpfs0/y.gusak/pretrained/DCP/imagenet/models_torch_1-0-1/

resnet18_pruned0.3.pth	resnet18_pruned0.7.pth	resnet50_pruned0.5.pth
resnet18_pruned0.5.pth	resnet50_pruned0.3.pth	resnet50_pruned0.7.pth


In [14]:
from collections import OrderedDict, defaultdict

import torch
import numpy as np

import sys
sys.path.append('/trinity/home/y.gusak/ReducedOrderNet/src')
from models.dcp.pruned_resnet import PrunedResNet

sys.path.append('../')
from flopco import FlopCo
import dataloaders

## DCP models

In [3]:
def load_dcpmodel(initialdcpmodels_path, depth, pruning_rate):
    model_name = 'resnet{}dcppruned{}_imagenet'.format(depth, pruning_rate)
    model = PrunedResNet(depth=depth, pruning_rate = pruning_rate, num_classes=1000)
    model_weights_path = "{}/resnet{}_pruned{}.pth".format(initialdcpmodels_path, depth, pruning_rate)                     
    model.eval()
    model_dict = model.state_dict() 
    pretrained_dict = torch.load(model_weights_path, map_location = 'cpu')
    model_dict.update(pretrained_dict)
    model.load_state_dict(model_dict)
    return model

def get_dcp_flops(initialdcpmodels_path):
    
    flops = defaultdict()
    
    for depth in [18, 50]:
        for pruning_rate in [0.3, 0.5, 0.7]:
            model_name = 'resnet{}dcppruned{}_imagenet'.format(depth, pruning_rate)
            
            m = load_dcpmodel(initialdcpmodels_path, depth, pruning_rate)
            
            flopco_m = FlopCo(m)
            flops[model_name] = flopco_m.total_flops
    
    return flops   

In [9]:
initialdcpmodels_path = "/gpfs/gpfs0/y.gusak/pretrained/DCP/imagenet"

depth = 50
pruning_rate = 0.5

model = load_dcpmodel(initialdcpmodels_path, depth, pruning_rate)

In [10]:
import sys
sys.path.append('../')
from model_utils import get_layer_names

layer_names, conv_layer_mask = get_layer_names(model)


fc_layer_mask = (1 - conv_layer_mask).astype(bool)

print(layer_names[conv_layer_mask][::-1])
print(layer_names[fc_layer_mask])

['layer4.2.conv3' 'layer4.2.conv2' 'layer4.2.conv1' 'layer4.1.conv3'
 'layer4.1.conv2' 'layer4.1.conv1' 'layer4.0.downsample.0'
 'layer4.0.conv3' 'layer4.0.conv2' 'layer4.0.conv1' 'layer3.5.conv3'
 'layer3.5.conv2' 'layer3.5.conv1' 'layer3.4.conv3' 'layer3.4.conv2'
 'layer3.4.conv1' 'layer3.3.conv3' 'layer3.3.conv2' 'layer3.3.conv1'
 'layer3.2.conv3' 'layer3.2.conv2' 'layer3.2.conv1' 'layer3.1.conv3'
 'layer3.1.conv2' 'layer3.1.conv1' 'layer3.0.downsample.0'
 'layer3.0.conv3' 'layer3.0.conv2' 'layer3.0.conv1' 'layer2.3.conv3'
 'layer2.3.conv2' 'layer2.3.conv1' 'layer2.2.conv3' 'layer2.2.conv2'
 'layer2.2.conv1' 'layer2.1.conv3' 'layer2.1.conv2' 'layer2.1.conv1'
 'layer2.0.downsample.0' 'layer2.0.conv3' 'layer2.0.conv2'
 'layer2.0.conv1' 'layer1.2.conv3' 'layer1.2.conv2' 'layer1.2.conv1'
 'layer1.1.conv3' 'layer1.1.conv2' 'layer1.1.conv1'
 'layer1.0.downsample.0' 'layer1.0.conv3' 'layer1.0.conv2'
 'layer1.0.conv1' 'conv1']
['fc']


In [11]:
if depth == 18:
    filtered_layer_names = list(filter(lambda l: 'downsample' not in l\
                                       and 'layer' in l,\
                                       layer_names[conv_layer_mask][::-1]
                                      ))
    filtered_layer_names = np.array(filtered_layer_names)
    
    gr_schedules = [
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 4)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 8)]]
    
elif depth == 50:
    filtered_layer_names = list(filter(lambda l: 'downsample' not in l\
                                   and 'layer' in l\
                                   and 'conv2' in l,\
                                   layer_names[conv_layer_mask][::-1]
                                  ))
    filtered_layer_names = np.array(filtered_layer_names)
    
    gr_schedules = [
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 4)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, 8)],
        [' '.join(list(gr)) for gr in np.split(filtered_layer_names, [2, 8, 12])]
    ]

In [12]:
gr_schedules

[['layer4.2.conv2 layer4.1.conv2 layer4.0.conv2 layer3.5.conv2',
  'layer3.4.conv2 layer3.3.conv2 layer3.2.conv2 layer3.1.conv2',
  'layer3.0.conv2 layer2.3.conv2 layer2.2.conv2 layer2.1.conv2',
  'layer2.0.conv2 layer1.2.conv2 layer1.1.conv2 layer1.0.conv2'],
 ['layer4.2.conv2 layer4.1.conv2',
  'layer4.0.conv2 layer3.5.conv2',
  'layer3.4.conv2 layer3.3.conv2',
  'layer3.2.conv2 layer3.1.conv2',
  'layer3.0.conv2 layer2.3.conv2',
  'layer2.2.conv2 layer2.1.conv2',
  'layer2.0.conv2 layer1.2.conv2',
  'layer1.1.conv2 layer1.0.conv2'],
 ['layer4.2.conv2 layer4.1.conv2',
  'layer4.0.conv2 layer3.5.conv2 layer3.4.conv2 layer3.3.conv2 layer3.2.conv2 layer3.1.conv2',
  'layer3.0.conv2 layer2.3.conv2 layer2.2.conv2 layer2.1.conv2',
  'layer2.0.conv2 layer1.2.conv2 layer1.1.conv2 layer1.0.conv2']]

In [13]:
model

PrunedResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): PrunedBottleneck(
      (conv1): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(32, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1